In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import rasterio
from shapely.geometry import Point
plt.rcParams.update({"pdf.fonttype":42})

# Start here if you have new data

In [ ]:
results_p = '../dem_tiles_agu'
lsdtt_results = [file for file in Path(results_p).glob('*HilltopData.csv')]
# lsdtt_results

In [ ]:
data_frames = []
for result in lsdtt_results:
    df =pd.read_csv(result)
    df['filename'] = result.stem
    df['tile_number'] = int('_'.join(result.stem.split('_')[1:-2]))
    data_frames.append(df)
lsdtt_results_df = pd.concat(data_frames, axis=0).reset_index().rename(columns={'index':'original_index'})
lsdtt_results_df = lsdtt_results_df[lsdtt_results_df['Cht'] != -9999.000000]


In [ ]:
gdf = gpd.read_file('tiles/tiles_3dep_realigned.shp')
gdf['FID'] = gdf.index
gdf['centroid_x'] = gdf.centroid.geometry.x
gdf['centroid_y'] = gdf.centroid.geometry.y
gdf['EPSG'] = (32700-(np.round((45+gdf['centroid_y'])/90,0)*100)+np.round((183+gdf['centroid_x'])/6,0)).astype(int)
# gdf

In [ ]:
d = dict(zip(gdf['FID'],gdf['EPSG']))
lsdtt_results_df['EPSG'] = lsdtt_results_df['tile_number'].map(d)
# lsdtt_results_df

In [ ]:
lsdtt_results_df['latlong'] = np.nan

for code in lsdtt_results_df['EPSG'].unique():
    code_subset = lsdtt_results_df.loc[lsdtt_results_df['EPSG'] == code]

    code_subset_gdf = gpd.GeoDataFrame(
    code_subset, 
    geometry=gpd.points_from_xy(
        code_subset['easting'], code_subset['northing']), 
        #   crs=f'epsg:{code}' 
          crs=f'epsg:32617' 
          ).to_crs('EPSG:4326')
    lsdtt_results_df.loc[lsdtt_results_df['EPSG'] == code, 'latlong'] = code_subset_gdf.geometry

# gdf
# ak_subset_squares = gpd.GeoDataFrame(
#     ak_subset.drop('geometry', axis=1).copy(), geometry=ak_subset.square_geom, crs="EPSG:4326"
#     )

lsdtt_results_gdf = gpd.GeoDataFrame(
    lsdtt_results_df, 
    geometry=lsdtt_results_df['latlong'],
    crs="EPSG:4326"
)

lsdtt_results_gdf['latitude'] = lsdtt_results_gdf.geometry.y
lsdtt_results_gdf['longitude'] = lsdtt_results_gdf.geometry.x

In [ ]:
medians = lsdtt_results_gdf.drop(['latlong', 'geometry'], axis=1).groupby('filename').median().reset_index()

# Start here if you have the .csv already

In [ ]:
medians = pd.read_csv('data/lsdtt_tile_medians.csv')

## Extract climate data to tiles

In [ ]:
medians_gdf = gpd.GeoDataFrame(
    medians, 
    geometry=gpd.points_from_xy(
        medians['longitude'], medians['latitude']), # here we specify which columns have the long and lat
          crs='epsg:4326' # finally end specifying the WGS84 spatial reference
          )

In [ ]:
climate_df = medians_gdf.filter(['tile_number']).astype(int)

coord_list = [(x, y) for x, y in zip(medians_gdf["geometry"].x, medians_gdf["geometry"].y)]

p = Path('maps/CCSM/22000BP')

scr_list = [file for file in p.glob('an_*.tif')]

for srcpath in scr_list:
    src = rasterio.open(srcpath)
    climate_df[srcpath.stem] = [x[0] for x in src.sample(coord_list)]

p = Path('maps/PRISM')

scr_list = [file for file in p.glob('*.bil')]

for srcpath in scr_list:
    src = rasterio.open(srcpath)
    climate_df[srcpath.stem] = [x[0] for x in src.sample(coord_list)]


p = Path('maps/PRISM')

scr_list = [file for file in p.glob('*.bil')]

for srcpath in scr_list:
    src = rasterio.open(srcpath)
    att = srcpath.stem.split('_')[1]
    att_column = f'PRISM_{att}'
    climate_df[att_column] = [x[0] for x in src.sample(coord_list)]


In [ ]:
lsdtt_results_gdf = gpd.GeoDataFrame(
    pd.merge(lsdtt_results_gdf, climate_df, how='left', on='tile_number'), 
    geometry=lsdtt_results_gdf['latlong'],
    crs="EPSG:4326"
)

geology = gpd.read_file('maps/all_data.shp')
# joined2 = gpd.sjoin(lsdtt_results_gdf, geology, how='left')

# geology dissolve

geol_dissolve = geology.dissolve(by='unit_name').reset_index()
joined2 = gpd.sjoin(lsdtt_results_gdf, geol_dissolve, how='left')

In [ ]:
tuscarora = joined2[joined2['unit_name'].notna()][joined2[joined2['unit_name'].notna()]['unit_name'].str.contains("Tuscarora")]
# nontusc = joined2[joined2['unit_name'].notna()][~joined2[joined2['unit_name'].notna()]['unit_name'].str.contains("Tuscarora")]
# tuscarora


In [ ]:
# tuscarora.select_dtypes(exclude=['object']).drop(columns='geometry').groupby('tile_number').median().to_csv("output_data/tuscarora_tile_medians.csv")
